# Life cycle assessment

For a sustainable bioeconomy, it is critical to assess the environmental impact of a production process. In this tutorial, we will conduct a cradle-to-biorefinery-gate LCA of a sugarcane biorefinery that produces electricity and ethanol. We focus on the global warning potential (GWP) because it is central to policy implementation.

![sugarcane_biorefinery](sugarcane_light.png)

## Example

A biorefinery system defines the life cycle inventory. Apart from the model itself, only the characterization factors for inputs and outputs are needed to perform LCA. These can be retrieved from standard public models like the Greenhouse Gases, Regulated Emissions, and Energy Use in Transportation (Argonne GREET Model). Here we load a sugarcane to ethanol biorefinery, define characterization factors from GREET, and perform LCA using energy based, revenue base, and displacement allocation methods (i.e. system expansion).

Define the impact indicator and the characterization factors:

In [2]:
### Load system
import biosteam as bst
from warnings import filterwarnings; filterwarnings('ignore') 
from biorefineries import cane
sc = cane.Biorefinery('S1') # Sugarcane conventional ethanol
system = sc.system

### Set characterization factors on a per kg basis, as available in GREET 2020

# Key for GWP characterization factor; we make it informative
# but any value can work, even just a number.
# Note that the impact assessment methodology used in GREET is the
# Tool for the Reduction and Assessment of Chemical and other 
# Environmental Impacts (TRACI)
GWP = 'GWP 100yr' 
bst.settings.define_impact_indicator(key=GWP, units='kg*CO2e')

# Sugarcane for ethanol production, adjusted for moisture content (0.75 originally, 0.70 in the model).
sc.sugarcane.set_CF(GWP, 0.02931 * 0.30 / 0.25, basis='kg', units='kg*CO2e') 

# Production of phosphoric acid from P2O5
sc.H3PO4.set_CF(GWP, 1.) # Basis defaults to kg; units default to defined units

# NG-Fired Simple-Cycle Gas Turbine CHP Plant, no transmission included
bst.settings.set_electricity_CF(GWP, 0.36, basis='kWhr', units='kg*CO2e')

# Lime production from lime stone, adjusted for dilution
lime_dilution = 1 - sc.lime.get_mass_composition('Water')
sc.lime.set_CF(GWP, 1.28 * lime_dilution)

# Gasoline blendstock from Crude Oil for US Refineries
sc.denaturant.set_CF(GWP, 0.84)

# Assume all other feeds are negligible.

Compute the GWP per kg of ethanol using displacement and energy allocation:

In [5]:
# Displacement allocation [kg-CO2e / kg-ethanol]
ethanol_production = system.get_mass_flow(sc.ethanol) 
GWP_ethanol_displacement_per_kg = system.get_net_impact(key=GWP, displace=True) / ethanol_production

# Energy allocation by gasoline gallon equivalent (GGE)
allocation_factors = system.get_property_allocation_factors(
    name='energy', products={sc.ethanol}
) # kg-CO2e / GGE

GWP_total = system.get_net_impact(key=GWP, displace=False)

GWP_ethanol_energy = (
    GWP_total * allocation_factors['Advanced ethanol'] 
) # kg-CO2e / yr

GWP_ethanol_energy_per_kg = GWP_ethanol_energy / ethanol_production # kg-CO2e / kg sugarcane ethanol

print(
    "Cradle to biorefinery gate GWP of sugarcane ethanol:\n "
    f"Displacement allocation: {GWP_ethanol_displacement_per_kg:.2f} [kg CO2e / kg]\n "
    f"Energy allocation: {GWP_ethanol_energy_per_kg:.2f} [kg CO2e / kg]\n "
)

Cradle to biorefinery gate GWP of sugarcane ethanol:
 Displacement allocation: -0.57 [kg CO2e / kg]
 Energy allocation: 0.37 [kg CO2e / kg]
 


Note that biogenic emissions do not contribute any GWP while that non-biogenic emissions associated to the bioreinfery do. In this example, all emissions are biogenic. The GWP computed here using energy allocation is very close to the value available in the [Ecoinvent life cycle inventory](https://ecoinvent.org/) (2020 database) for a modern autonomous sugarcane ethanol plant in Brazil (value not shown here to avoid proprietary issues).

## Life cycle inventory and impact breakdown

A breakdown of the inventory and impacts comes in handy for inpecting values and verifying results. The `biosteam.report` module includes a few helpful functions for this:

In [6]:
from biosteam import report
report.lca_inventory_table(
    systems=[system], 
    key=GWP,
    items=[sc.ethanol], # For including products without characterization factors
)

Inventory [kg/yr]
Inputs  H3PO4                          3.43e+05
        Denaturant                     2.41e+06
        Lime                           6.86e+05
        Sugarcane                       1.6e+09
Outputs Advanced ethanol               1.12e+08
        Electricity [kWhr/yr]          3.39e+08

In [7]:
report.lca_displacement_allocation_table(
    systems=[system], 
    key=GWP,
    items=[sc.ethanol], # For dividing yearly impact by ethanol production  
)

Characterization factor [kg*CO2e/kg] GWP 100yr [kg*CO2e/kg*advanced ethanol]
Inputs                  H3PO4                                         1                              0.00306    
                        Denaturant                                 0.84                               0.0181    
                        Lime                                     0.0589                             0.000361    
                        Sugarcane                                0.0352                                0.503    
Total inputs                                                                                           0.524    
Outputs displaced       Electricity                   0.36 kg*CO2e/kWhr                                 1.09    
Total outputs displaced                                                                                 1.09    
Total                                                                                                 -0.566

In [10]:
report.lca_property_allocation_factor_table(
    systems=[system],
    property='energy',
    products=[sc.ethanol],
)

,Energy allocation factors
Advanced ethanol,0.712
Electricity,0.288


In [11]:
report.lca_displacement_allocation_factor_table(
    systems=[system],
    items=[sc.ethanol],
    key=GWP,
)

,Displacement allocation factors
Advanced ethanol,-1.08
Electricity,2.08


## Problem
Compute the GWP per kg of ethanol using revenue allocation:

In [3]:
# Hint: here is a list of allocation methods with units
bst.settings.allocation_properties

{'energy': 'kJ', 'revenue': 'USD', 'mass': 'kg'}

In [13]:
# Economic/revenue allocation
allocation_factors = system.get_property_allocation_factors(
    name='energy', products={sc.ethanol}
) # kg-CO2e / GGE

GWP_total = system.get_net_impact(key=GWP, displace=False) # kg-CO2e / yr

GWP_ethanol_energy = (
    GWP_total * allocation_factors['Advanced ethanol'] 
) # kg-CO2e / yr

ethanol_production = system.get_mass_flow(sc.ethanol) # kg-ethanol / yr

GWP_ethanol_energy_per_kg = GWP_ethanol_energy / ethanol_production # kg-CO2e / kg sugarcane ethanol

print(
    "Cradle to biorefinery gate GWP of sugarcane ethanol:\n "
    f"Revenue allocation: {GWP_ethanol_energy_per_kg:.2f} [kg CO2e / kg]\n "
)

Cradle to biorefinery gate GWP of sugarcane ethanol:
 Revenue allocation: 0.37 [kg CO2e / kg]
 
